In [33]:
import torch
import torch.nn as nn
from torchvision import models
import warnings
warnings.filterwarnings("ignore") 

In [8]:
class Yolov1(nn.Module):
    def __init__(self,S=7, B=2, C=20):
        super().__init__()
        resnet = models.resnet152(pretrained=True)
        self.backbone = nn.Sequential(*list(resnet.children())[:-2]) 

        # YOLO Detection Head (Example)
        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2048*7*7, 4096),
            nn.Dropout(0.0),
            nn.LeakyReLU(0.1),
            nn.Linear(4096, S*S*(C+B*5))
        )
    def forward(self,x):
        x = self.backbone(x)
        x = self.head(x)
        return x
    

In [9]:
x = torch.rand(1,3,224,224)
yolo = Yolov1()
yolo(x).shape

torch.Size([1, 1470])

#=============================================================================================#

In [10]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, bn_act = True, **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias= not bn_act, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.LeakyReLU(0.1)
        self.use_bn_act = bn_act

    def forward(self, x):
        if self.use_bn_act:
            return self.act(self.bn(self.conv(x)))
        else:
            return self.conv(x)             
        

In [18]:
class ScalePrediction(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.head = nn.Sequential(
            CNNBlock(in_channels, 2*in_channels,kernel_size=3, padding=1),
            CNNBlock(2*in_channels, (num_classes +5) *3, bn_act=False, kernel_size=1), #[po, x, y, w,h]
        )
        self.num_classes = num_classes
    def forward(self,x):
        x = self.head(x)
        x = x.reshape(x.shape[0], 3, self.num_classes + 5, x.shape[2], x.shape[3])
        x = x.permute(0,1,3,4,2)
        return x

In [22]:
conv = CNNBlock(3,64,kernel_size=3)
conv(x).shape

torch.Size([1, 64, 222, 222])

In [23]:
class Yolov3(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = models.resnet152(pretrained=True)
        self.backbone = nn.Sequential(*list(resnet.children())[:-2]) 

        # YOLO Detection Head (Example)
        self.head = ScalePrediction(2048, 20)
    def forward(self,x):
        x = self.backbone(x)
        x = self.head(x)
        return x

In [24]:
x = torch.rand(1,3,224,224)
yolo = Yolov3()
yolo(x).shape

torch.Size([1, 3, 7, 7, 25])

#==================================================================================================#

In [ ]:
class DFL(nn.Module):
    """
    Integral module of Distribution Focal Loss (DFL).

    Proposed in Generalized Focal Loss https://ieeexplore.ieee.org/document/9792391
    """

    def __init__(self, c1=16):
        """Initialize a convolutional layer with a given number of input channels."""
        super().__init__()
        self.conv = nn.Conv2d(c1, 1, 1, bias=False).requires_grad_(False)
        x = torch.arange(c1, dtype=torch.float)
        self.conv.weight.data[:] = nn.Parameter(x.view(1, c1, 1, 1))
        self.c1 = c1

    def forward(self, x):
        """Applies a transformer layer on input tensor 'x' and returns a tensor."""
        b, _, a = x.shape  # batch, channels, anchors
        return self.conv(x.view(b, 4, self.c1, a).transpose(2, 1).softmax(1)).view(b, 4, a)
        # return self.conv(x.view(b, self.c1, 4, a).softmax(1)).view(b, 4, a)

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque

In [26]:
memory = deque(maxlen=10000)

In [27]:
memory

deque([], maxlen=10000)

In [31]:
memory.extend([1,2,3])
memory

deque([10, 10, 11, 1, 2, 3], maxlen=10000)

In [32]:
type(memory)

collections.deque